# Test Parallel Tool Use of Claude

Claude can call multiple tool in parallel by default.

In [2]:
from dotenv import load_dotenv
load_dotenv()

import anthropic

client = anthropic.Anthropic()

def get_weather(city: str) -> str:
    """Get the weather in a given city"""
    return f"The weather in {city} is sunny."

def get_air_quality(city: str) -> str:
    """Get the air quality in a given city"""
    return f"The air quality in {city} is good."

get_weather_metadata = {
    "name": "get_weather",
    "description": "Get the weather in a given city",
    "input_schema": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to get the weather for"
            }
        },
        "required": ["city"]
    }
}

get_air_quality_metadata = {
    "name": "get_air_quality",
    "description": "Get the air quality in a given city",
    "input_schema": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to get the air quality for"
            }
        },
        "required": ["city"]
    }
}

tools = [get_weather_metadata, get_air_quality_metadata]

In [3]:
messages = [
  {
    "role": "user",
    "content": "What is the weather and air quality in Beijing?"
  }
]

message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024, # max_tokens is the maxium tokens to generate, it is required
    messages=messages,
    tools=tools
)

In [4]:
# Get the text output from the message
content = message.content
text = content[0].text
print(text)

I'll get both the weather and air quality information for Beijing.


In [5]:
# Get the tool use from the message
if message.stop_reason == "tool_use":
    for block in message.content:
        if block.type == "tool_use":
            print(block)
else:
    print("No tool use")

ToolUseBlock(id='toolu_01Qae87WtdrgRxuoom9PChwT', input={'city': 'Beijing'}, name='get_weather', type='tool_use')
ToolUseBlock(id='toolu_01FLB5cmiN9qRBCGKiGBGrw8', input={'city': 'Beijing'}, name='get_air_quality', type='tool_use')
